## cell_seg

The main parameters of the program include,
- Input

    - ```is_gpu```: Use GPU or not<br>
    - ```method```: segmentation methods, ['deepcell', 'cellpose', 'stereocell', 'sam', 'lt']<br>
    - ```image_path```: Stereo-seq Image data<br>

- Output

    - ```output_path```: result of cell segmentation

In [2]:
import os
import time
import subprocess
import cv2

work_path = os.path.abspath('.')
# work_path = '/data/work/benchmark/benchmark'
__py__ = {
    'MEDIAR':'/media/Data/shican/software/miniconda3/envs/MEDIAR/bin/python',
    'cellpose': 'python',
    'cellpose3':'python',
    'deepcell': 'python',
    'sam': 'python',
    'stardist':'python',
    'cellprofiler':'/media/Data/shican/software/miniconda3/envs/cp4/bin/python'
}
__methods__ = ['MEDIAR','cellpose','cellpose3', 'sam','stardist','deepcell','cellprofiler']


__script__ = {
    'MEDIAR':os.path.join(work_path,'src/methods/MEDIAR/mediar_main.py'),
    'cellpose': os.path.join(work_path, 'src/methods/cellpose_main.py'),
    'cellpose3':os.path.join(work_path,'src/methods/cellpose3_main.py'),
    'deepcell': os.path.join(work_path, 'src/methods/deepcell_main.py'),
    'sam': os.path.join(work_path, 'src/methods/sam_main.py'),
    'stardist':os.path.join(work_path,'src/methods/stardist_main.py'),
    'cellprofiler':os.path.join(work_path,'src/methods/cellprofiler/cellprofiler_main.py')
}
def generate_grayscale_negative(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    negative_img = cv2.bitwise_not(img)
    cv2.imwrite(output_path, negative_img)

def generate_negative(image_path, output_path):
    img = cv2.imread(image_path)
    negative_img = cv2.bitwise_not(img)
    cv2.imwrite(output_path, negative_img)

def process_images_in_directory(image_dir, new_dir, img_type):
    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)
        if os.path.isfile(image_path):
            new_image_path = os.path.join(new_dir, filename)
            if img_type == 'he':
                generate_grayscale_negative(image_path, new_image_path)
            elif img_type == 'mif':
                generate_negative(image_path, new_image_path)
    return new_dir
####### Image names must include '-img', and mask image names must include '-mask'
####### The part you need to modify #####
is_gpu = True
method = ['MEDIAR','cellpose','cellpose3']
image_dir = '/media/Data/shican/data/benchmark_demo/ssDNA/input/'
output_path = '/media/Data/shican/data/benchmark_demo/ssDNA/output/'
img_type = 'ss' # he or ss or mif or dapi
###########################
print(os.listdir(image_dir))
print(work_path)

processed_dir = image_dir
assert os.path.isdir(image_dir), "Input path must be a directory"
print(f'get {len(os.listdir(image_dir))} files')
if img_type == 'he' or img_type == 'mif':
    new_dir = os.path.join(os.path.dirname(image_dir), 'processed_images')
    os.makedirs(new_dir, exist_ok=True)
    processed_dir = process_images_in_directory(image_dir, new_dir, img_type)

for m in method:
    assert m in __methods__

for m in method:
    start = time.time()
    if m == 'cellprofiler':
        cmd = '{} {} -i {} -o {} -g {} -t {}'.format(
        __py__[m], __script__[m], image_dir, os.path.join(output_path, m), is_gpu, img_type)
    elif m == 'stardist' and img_type == 'he':
        cmd = '{} {} -i {} -o {} -g {} -t {}'.format(
        __py__[m], __script__[m], image_dir, os.path.join(output_path, m), is_gpu, img_type)
    elif m == 'cellpose' or  m == 'cellpose3' or m == 'MEDIAR' and img_type == 'mif':
        cmd = '{} {} -i {} -o {} -g {} -t {}'.format(
        __py__[m], __script__[m], image_dir, os.path.join(output_path, m), is_gpu, img_type)
    else:
        cmd = '{} {} -i {} -o {} -g {} -t {}'.format(
            __py__[m], __script__[m], processed_dir, os.path.join(output_path, m), is_gpu, img_type)
    os.system(cmd)
    t = time.time() - start
    print('{} ran for a total of {} s'.format(m, t))
    print('{} result saved to {}'.format(m, os.path.join(output_path, m)))    

['U2FsdGVkX1StP6lMBlfLr7MIIvY938tUcR5Ao1oE-x14530_y7678_w256_h256-img.tif']
/media/Data/shican/code/cs-benchmark2
get 1 files
Prediction finished: U2FsdGVkX1StP6lMBlfLr7MIIvY938tUcR5Ao1oE-x14530_y7678_w256_h256-img.tif; img size = torch.Size([1, 3, 256, 256]); costing: 2.24s

 Total Time Cost: 2.24s
MEDIAR ran for a total of 19.612496614456177 s
MEDIAR result saved to /media/Data/shican/data/benchmark_demo/ssDNA/output/MEDIAR


/media/Data/shican/software/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/media/Data/shican/software/miniconda3/lib/python3.9/site-packages/cellpose/resnet_torch.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det

cellpose ran for a total of 6.724498510360718 s
cellpose result saved to /media/Data/shican/data/benchmark_demo/ssDNA/output/cellpose


/media/Data/shican/software/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/media/Data/shican/software/miniconda3/lib/python3.9/site-packages/cellpose/resnet_torch.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det

cellpose3 ran for a total of 6.63588285446167 s
cellpose3 result saved to /media/Data/shican/data/benchmark_demo/ssDNA/output/cellpose3


## Evaluation

In [3]:
import os

py = 'python'
script = 'src/eval/cell_eval_multi.py'

gt_path = '/media/Data/shican/data/DAPI/gt/'    # Ground Truth
dt_path = '/media/Data/shican/data/DAPI/output/'    # Result of the algorithm's prediction
eval_path = '/media/Data/shican/data/DAPI/eval/'  # Evaluate the output of the results
if not os.path.exists(eval_path): os.makedirs(eval_path)

cmd = '{} {} -g {} -d {} -o {}'.format(py, script, gt_path, dt_path, eval_path)
print(cmd)
os.system(cmd)



python src/eval/cell_eval_multi.py -g /media/Data/shican/data/DAPI/gt/ -d /media/Data/shican/data/DAPI/output/ -o /media/Data/shican/data/DAPI/eval/
Namespace(dt_path='/media/Data/shican/data/DAPI/output/', func=<function main at 0x7f3ddc5acaf0>, gt_path='/media/Data/shican/data/DAPI/gt/', output_path='/media/Data/shican/data/DAPI/eval/') []
dataset_name:gt
methods:['cellprofiler', 'stardist', 'MEDIAR', 'cellpose', 'deepcell', 'sam', 'cellpose3']
gt:/media/Data/shican/data/DAPI/gt/
dt:/media/Data/shican/data/DAPI/output/cellprofiler


Load data stardist:  13%|█▎        | 40/303 [00:00<00:00, 394.20it/s]

gt:/media/Data/shican/data/DAPI/gt/
dt:/media/Data/shican/data/DAPI/output/stardist


Load data MEDIAR:  13%|█▎        | 39/303 [00:00<00:00, 382.72it/s]

gt:/media/Data/shican/data/DAPI/gt/
dt:/media/Data/shican/data/DAPI/output/MEDIAR


...

Load data cellpose3:  14%|█▎        | 41/303 [00:00<00:00, 406.68it/s]

gt:/media/Data/shican/data/DAPI/gt/
dt:/media/Data/shican/data/DAPI/output/cellpose3


100%|██████████| 303/303 [00:14<00:00, 21.47it/s]

{'cellprofiler': {'precision': 0.6624948912432086, 'recall': 0.5509899444791634, 'f1': 0.5903575145430213, 'jaccard': 0.5893533072383649, 'dice': 0.7324995564772128}, 'stardist': {'precision': 0.736951321608616, 'recall': 0.7059010355984056, 'f1': 0.7100787185491375, 'jaccard': 0.6177970580631963, 'dice': 0.754172497260609}, 'MEDIAR': {'precision': 0.8831493287454986, 'recall': 0.7658951315904489, 'f1': 0.8139754697904806, 'jaccard': 0.6722152428882462, 'dice': 0.7931492201331234}, 'cellpose': {'precision': 0.9221180861636038, 'recall': 0.671288953476303, 'f1': 0.7644684267326598, 'jaccard': 0.6839520892969733, 'dice': 0.8019311610735222}, 'deepcell': {'precision': 0.8339642675290757, 'recall': 0.8329196530974421, 'f1': 0.8253691828784488, 'jaccard': 0.7368138626749284, 'dice': 0.8432473120073369}, 'sam': {'precision': 0.6768262577708583, 'recall': 0.7188770431382415, 'f1': 0.6837189322350743, 'jaccard': 0.6013997454510731, 'dice': 0.7435216957129422}, 'cellpose3': {'precision': 0.8046

0